In [ ]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

In [2]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

In [5]:
GROQ_API_KEY="GROQ_API"

In [6]:
def print_res(response):
  response_txt=response["result"]
  for chunk in response_txt.split("\n"):
    if not chunk:
      print()
      continue
      print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))


In [11]:
!mkdir data
!gdown 1wvPxH654nsp78ANiSVEbmLt2LMSB8ph3 -O "Prompt_Engg/Prompt-Engineering-Lecture-Elvis.pdf"

mkdir: cannot create directory ‘data’: File exists
/usr/local/lib/python3.10/dist-packages/gdown/download.py:32: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs4.BeautifulSoup(line, features="html.parser")
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1wvPxH654nsp78ANiSVEbmLt2LMSB8ph3

but Gdown can't. Please check connections and permissions.


In [13]:
!gdown https://drive.google.com/file/d/14hofe6tWLJWTeIH4aoPcqzmmatEopAdF/view?usp=drive_link

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=14hofe6tWLJWTeIH4aoPcqzmmatEopAdF
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/14hofe6tWLJWTeIH4aoPcqzmmatEopAdF/view?usp=drive_link
To: /content/view?usp=drive_link
92.5kB [00:00, 3.85MB/s]


In [16]:
instruction="""
The provided document is a guide of Prompt Engineering.
try to be accurate while answering the questions. Act and keep the tone like a Professional Prompt Engineer.
"""

parser = LlamaParse(
    api_key="LLAMA_PARSE API",
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents= await parser.aload_data("/content/Prompt-Engineering-Lecture-Elvis.pdf")

Started parsing the file under job_id c154137e-c601-428d-bc9b-30f46edac94c


In [17]:
parsed_doc=llama_parse_documents[0]

In [19]:
Markdown(parsed_doc.text[:7096])

# Prompt Engineering

A lecture by DAIR.AI

Elvis Saravia
---
# Prerequisites & Objectives

Prerequisites:PythonKnowledge of language modelsBasic understanding of deep learning / ML concepts

ObjectivesPresent an introduction to prompt engineeringPresent an overview of the latest prompting techniquesProvide demonstrations and exercises to practice different prompting techniques
---
# Agenda

- Introduction to Prompt Engineering
- Advanced Techniques for Prompt Engineering
- Applications & Tools
- Conclusion & Future Directions
---
Part 1

Introduction to Prompt Engineering
---
What are prompts?

- Prompts involve instructions and context passed to a language model to achieve a desired task
- Prompt engineering is the practice of developing and optimizing prompts to efficiently use language models (LMs) for a variety of applications
- Prompt engineering is a useful skill for AI engineers and researchers to improve and efficiently use language models

What is prompt engineering?

Prompt engineering is an area used to process and guide the creation of a set of prompts, or questions, that lead the user toward a desired outcome. It is an effective tool for designers to create intuitive user experiences that are easy to use. This method is often used by users in interactive design and software development, as it allows them to easily understand how to interact with a system or product.
---
# Why Prompt Engineering?

- Why learn prompt engineering?
- Important for research, discoveries, and advancement
- Helps to test and evaluate the limitations of LLMs
- Enables all kinds of innovative applications on top of LLMs

|ANTHROPIC|Prompt Engineer and Librarian|APPLY FOR THIS JOB| |
|---|---|---|---|
|SAN FRANCISCO CA|PRODUCT|FULL-TIME|HYBRID|

Anthropic's mission is to create reliable, interpretable, and steerable AI systems. We want AI to be safe for our customers and for society as a whole.

Anthropic's AI technology is amongst the most capable and safe in the world. However, large language models are a new type of intelligence, and the art of instructing them in a way that delivers the best results is still in its infancy.

You will figure out the best methods of prompting our AI to accomplish tasks, then document these methods to build up a library of tools and a set of tutorials that allows others to learn prompt engineering or simply find prompts that would be ideal for them.

Compensation and Benefits*

Anthropic's compensation package consists of three elements: salary, equity, and benefits. We are committed to pay fairness and aim for these three elements collectively to be highly competitive with market rates:

- Salary: The expected salary range for this position is $250k - $335k.
- Equity: Equity will be a major component of the total compensation for this position. We aim to offer higher-than-average equity compensation for a company of our size and communicate equity amounts at the time of offer issuance.

Source: https://jobs.lever.co/Anthropic/e3cde481-d446-460f-b576-93cab67bd1ed
---
# First Basic PromptResponse

InstructionThe sky is blueThe sky is a beautiful blue color during the day. The blue of the sky is created by the sun’s light. The blue is a result of the scattering of air molecules in the atmosphere to reflect the shorter wavelength of blue light back to our eyes.

Model: “text-davinci-003”

temperature: 0.7

top-p: 1
---
# Elements of a Prompt

- Instructions
- Context
- Input data
- Output indicator

Classify the text into neutral, negative or positiveText: I think the food was okay.Sentiment:
---
Settings to keep in mind

- When prompting a language model you should keep in mind a few settings
- You can get very different results with prompts when using different settings
- One important setting is controlling how deterministic the model is when generating completion for prompts
- Temperature and top_p are two important parameters to keep in mind
- Generally, keep these low if you are looking for exact answers
- ...keep them high if you are looking for more diverse responses
---
# Designing Prompts for Different Tasks

In the next few slides, we will cover a few examples of common tasks using different prompts

Tasks covered:Text SummarizationQuestion AnsweringText ClassificationRole PlayingCode GenerationReasoning
---
# Text Summarization

|Context|Instruction|
|---|---|
|Antibiotics are a type of medication used to treat bacterial infections. They work by either killing bacteria or preventing them from reproducing, allowing the body’s immune system to fight off the infection.| |
|Antibiotics are usually taken orally in the form of capsules, pills, or liquid solutions, sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance.| |
|Explain the above in one sentence:| |
|Antibiotics are medications used to treat bacterial infections by either killing bacteria or stopping them from reproducing, but they are not effective against viruses and overuse can lead to antibiotic resistance.| |
---
# Question Answering

|Question|Answer|
|---|---|
|What was OKT3 originally sourced from?|Mice.|

Context: The company Teplizumab Orthotraces Pharmaceutical traces its roots to New Jersey scientists. The drug, originally an early version of the antibody called OKT3, was sourced from mice. In 1986, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. It was approved to help prevent organ rejection after kidney transplants, making it suitable for human use.

Context obtained from: https://www.nature.com/articles/d41586-023-00400-x
---
# Text Classification

Classify the text into neutral, negative or positive.

|Text:|I think the food was okay.|
|---|---|
|Sentiment:|Neutral|
---
# Role Playing

The following is a conversation with an AI research assistant. The tone is technical and scientific.

|Human:|Hello, who are you?|
|---|---|
|AI:|Greeting! I am an AI research assistant. How can I help you today?|
|Human:|Can you tell me about the creation of black holes?|
|AI:|Sure! Black holes are regions of spacetime where the gravitational force is so strong that nothing, not even light, can escape from its core. They are created when a very massive star dies and collapses in on itself, forming a singularity of infinite density. The intense gravity of the singularity pulls all matter and radiation around it, creating the black hole.|
---
# Code Generation

|Table|departments|columns|=|[DepartmentId, DepartmentName]|
|---|---|---|---|---|
|Table|students|columns|=|[DepartmentId, StudentId, StudentName]|

Create Sciencea DepartmentMySQL query for all students in the Computer

SELECT StudentId, StudentName

FROM students

WHERE DepartmentId DepartmentName IN (SELECT DepartmentId FROM departments WHERE = 'Computer Science');
---
# Reasoning

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. Solve by numbers, breaking the problem into steps. First, identify the odd numbers and indicate whether the result is

In [20]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

In [21]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)


16

In [23]:
print(docs[0].page_content)

Prompt Engineering

A lecture by DAIR.AI

Elvis Saravia

Prerequisites & Objectives

Prerequisites:PythonKnowledge of language modelsBasic understanding of deep learning / ML concepts

ObjectivesPresent an introduction to prompt engineeringPresent an overview of the latest prompting techniquesProvide demonstrations and exercises to practice different prompting techniques

Agenda

Introduction to Prompt Engineering

Advanced Techniques for Prompt Engineering

Applications & Tools

Conclusion & Future Directions

Part 1

Introduction to Prompt Engineering

What are prompts?

Prompts involve instructions and context passed to a language model to achieve a desired task

Prompt engineering is the practice of developing and optimizing prompts to efficiently use language models (LMs) for a variety of applications

Prompt engineering is a useful skill for AI engineers and researchers to improve and efficiently use language models

What is prompt engineering?

Prompt engineering is an area used

In [24]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [26]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [27]:
%%time
query="What is Prompt Engineering"
similar_docs=qdrant.similarity_search_with_score(query)

CPU times: user 292 ms, sys: 6.73 ms, total: 298 ms
Wall time: 307 ms


In [29]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:2560]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: Prompt Engineering

A lecture by DAIR.AI

Elvis Saravia

Prerequisites & Objectives

Prerequisites:PythonKnowledge of language modelsBasic understanding of deep learning / ML concepts

ObjectivesPresent an introduction to prompt engineeringPresent an overview of the latest prompting techniquesProvide demonstrations and exercises to practice different prompting techniques

Agenda

Introduction to Prompt Engineering

Advanced Techniques for Prompt Engineering

Applications & Tools

Conclusion & Future Directions

Part 1

Introduction to Prompt Engineering

What are prompts?

Prompts involve instructions and context passed to a language model to achieve a desired task

Prompt engineering is the practice of developing and optimizing prompts to efficiently use language models (LMs) for a variety of applications

Prompt engineering is a useful skill for AI engineers and researchers to improve and efficiently use language models

What is prompt engineering?

Prompt engineering is an are

In [30]:
%%time
retriever=qdrant.as_retriever(search_kwargs={"k":5})
retrieved_docs = retriever.invoke(query)

CPU times: user 395 ms, sys: 6.66 ms, total: 402 ms
Wall time: 407 ms


In [31]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:2560]}\n")
    print("-" * 80)
    print()


id: fd52024e78004f27bea95bb033d71acb

text: Prompt Engineering

A lecture by DAIR.AI

Elvis Saravia

Prerequisites & Objectives

Prerequisites:PythonKnowledge of language modelsBasic understanding of deep learning / ML concepts

ObjectivesPresent an introduction to prompt engineeringPresent an overview of the latest prompting techniquesProvide demonstrations and exercises to practice different prompting techniques

Agenda

Introduction to Prompt Engineering

Advanced Techniques for Prompt Engineering

Applications & Tools

Conclusion & Future Directions

Part 1

Introduction to Prompt Engineering

What are prompts?

Prompts involve instructions and context passed to a language model to achieve a desired task

Prompt engineering is the practice of developing and optimizing prompts to efficiently use language models (LMs) for a variety of applications

Prompt engineering is a useful skill for AI engineers and researchers to improve and efficiently use language models

What is prompt engi

In [32]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 80.9MiB/s]


In [33]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)


Running pairwise ranking..
CPU times: user 2.49 s, sys: 274 ms, total: 2.77 s
Wall time: 2.7 s


3

In [34]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()


id: fd52024e78004f27bea95bb033d71acb

text: Prompt Engineering

A lecture by DAIR.AI

Elvis Saravia

Prerequisites & Objectives

Prerequisites:PythonKnowledge of language modelsBasic understanding of deep learning / ML concepts

ObjectivesPresent an introduction to prompt engineeringPresent an overv

score: 0.9996650218963623
--------------------------------------------------------------------------------

id: ff715b2f781940e1b732236abbcdcba3

text: Join our Discord to discuss more about prompt engineering

Table of Contents

Guides

Papers

Tools & Libraries

Datasets

Blog: Guides, Tutorials, and Other Readings

Guides

The following are a set of guides on prompt engineering developed by us. Guides 

score: 0.9971957802772522
--------------------------------------------------------------------------------

id: 4603606863e34a0887bd69cbdacf040d

text: RLHF involves collecting high-quality prompt datasets

Popular examples:

Claude (Anthropic)

ChatGPT (OpenAI)

Step 1 Step 2 Step 3 Col

In [45]:
llm = ChatGroq(groq_api_key="GROQ_API",
               model_name="llama3-70b-8192",
               temperature=0)


In [46]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


In [47]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [49]:
%%time
response = qa.invoke("Why prompt engineering")


Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Prompt Engineering

A lecture by DAIR.AI

Elvis Saravia

Prerequisites & Objectives

Prerequisites:PythonKnowledge of language modelsBasic understanding of deep learning / ML concepts

ObjectivesPresent an introduction to prompt engineeringPresent an overview of the latest prompting techniquesProvide demonstrations and exercises to practice different prompting techniques

Agenda

Introduction to Prompt Engineering

Advanced Techniques for Prompt Engineering

Applications & Tools

Conclusion & Future Directions

Part 1

Introduction to Prompt Engineering

What are prompts?

Prompts involve instructions and context passed to a language model to achieve a desired task

Prompt engineering

In [51]:
print(response)

{'query': 'Why prompt engineering', 'result': '**Why Prompt Engineering?**\n\nPrompt engineering is important for research, discoveries, and advancement. It helps to test and evaluate the limitations of Large Language Models (LLMs) and enables innovative applications on top of LLMs.\n\nAdditionally, prompt engineering can be used not only to improve performance but also the reliability of responses from a safety perspective. It can help identify risky behaviors of LLMs, reduce harmful behaviors and risks that may arise from language models, and even aid in building safer AI systems.', 'source_documents': [Document(page_content="Prompt Engineering\n\nA lecture by DAIR.AI\n\nElvis Saravia\n\nPrerequisites & Objectives\n\nPrerequisites:PythonKnowledge of language modelsBasic understanding of deep learning / ML concepts\n\nObjectivesPresent an introduction to prompt engineeringPresent an overview of the latest prompting techniquesProvide demonstrations and exercises to practice different p